In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('whitegrid')
import matplotlib.pyplot as plt
%matplotlib inline
from numba import njit


import scipy
%load_ext snakeviz

In [156]:
from itertools import combinations
from sklearn.metrics import mean_absolute_error
from joblib import Parallel
from joblib import delayed

from src.data_management.utilities import load_testing_data
from src.data_management.utilities import load_training_data

## Candidate Implementations

In [2]:
def matrix_inversion_np(xTx, xTy):
    beta = np.linalg.inv(xTx).dot(xTy)
    return beta


def solve_np(xTx,xTy):
    beta = np.linalg.solve(xTx, xTy)
    return beta


def solve_scipy(xTx,xTy):
    beta = scipy.linalg.solve(xTx, xTy)
    return beta

def lu_solve_scipy(xTx, xTy):
    lu, piv = scipy.linalg.lu_factor(xTx)
    beta = scipy.linalg.lu_solve((lu, piv), xTy)
    return beta

## Putting it Together

In [3]:
def fit_related_ols_regressions(x, y, models, fit_method=matrix_inversion_np):
    """Fit multiple OLS regressions on the same dataset.
    
    Args:
        x (np.ndarray): 2d array with regressors
        y (pn.ndarray): 1d array with outcomes
        models (list): list of 1d integer arrays with indices 
            of included variables.
        
    Returns:
        estimates (list): List of parameter estimates. Same order as models.
        
    """
    y = y.reshape(-1, 1)
    xTx = x.T @ x
    xTy = x.T @ y
    estimates = []
    for mod in models:
        x_part = xTx[mod, mod]
        y_part = xTy[mod]
        estimates.append(fit_method(xTx[mod][:, mod], xTy[mod]))
    return estimates

In [87]:
def compute_error(coefficients, models, Xtest, ytest):
    """Compute mean absolute error of model predictions."""
    mae = np.empty(len(models))
    for i, coef in enumerate(coefficients):
        mae[i] = mean_absolute_error(
            ytest,
            Xtest[:, models[i]] @ coef
        )
    return mae

In [52]:
%%snakeviz
fit_related_ols_regressions(x, y, models, matrix_inversion_np)

 
*** Profile stats marshalled to file '/tmp/tmpdqbe16ll'. 
Embedding SnakeViz in this document...


In [134]:
def minimizer_k_subset(k, X, y, Xtest, ytest):
    """Find model that minimizes mae using k variables from n."""
    n = X.shape[1]
    models = choice_index(n, k)
    
    coefficients = fit_related_ols_regressions(X, y, models)
    
    mae = compute_error(coefficients, models, Xtest, ytest)
    
    minimizer = np.argmin(mae)
    return models[minimizer], mae[minimizer]

In [135]:
def choice_index(n, k):
    """Return all different possible k draws from n.
    
    Args:
        n (int): Number of elements in set.
        k (int): Number of element to draw from set.
    
    Returns:
        comb (list): List of tuples of indices, where
            each tuple represents one possible draw of
            k elements from the n elements.
    
    """
    index = np.arange(n)
    comb = list(combinations(index, k))
    comb = [np.array(e) for e in comb]
    return comb

In [131]:
X, y = load_training_data(nobs=10000)
XX = X.values
XX = np.insert(XX, 0, 1, axis=1)

Xtest, ytest = load_testing_data(nobs=1000)
XXtest = Xtest.values
XXtest = np.insert(XXtest, 0, 1, axis=1)

In [155]:
def to_parallelize(k):
    optimal_model, _ = minimizer_k_subset(k, XX, y, XXtest, ytest)
    return optimal_model

In [158]:
start = time.time()
optimal_models = {}

out = Parallel(n_jobs=2, prefer='processes')(
        delayed(to_parallelize) for k in [5, 20]
)

print(time.time() - start)

TypeError: cannot unpack non-iterable function object

In [141]:
optimal_models

{2: array([ 9, 21]),
 5: array([ 1,  8, 21, 22, 23]),
 23: array([ 0,  1,  2,  3,  5,  6,  7,  8,  9, 11, 12, 13, 14, 15, 16, 18, 19,
        20, 21, 22, 23, 26, 27])}